# 確率的な方策勾配法

参考：
* [Understanding the Effect of Stochasticity in Policy Optimization](https://arxiv.org/abs/2110.15572)
* [Beyond variance reduction: Understanding the true impact of baselines on policy optimization](https://arxiv.org/abs/2008.13773)

よくある方策勾配法の理論では基本的に「厳密な」勾配がある状況だけ考えてます．しかし，実際のアプリケーションでは確率的な勾配が使われます．
確率的な勾配になったときに何が言えるでしょうか？


## 厳密な方策の場合

[RL_PolicyGrad_softmax_convergence](RL_PolicyGrad_softmax_convergence.ipynb)と表記はほぼ同じなので省略します．

softmaxパラメータを考えます．
まずSoftmax PG, NPG, GNPGについてまとめてみましょう．

### Softmax PG

[RL_PolicyGrad_softmax_convergence](RL_PolicyGrad_softmax_convergence.ipynb)でやったので省略します．Gradient dominanceが成立し，そして適切な学習率のもとでは最適方策に$\Theta(1/t)$で収束します．

### NPG

* 更新方法：$\theta_{t+1} \leftarrow \theta_t+\eta \cdot r$
* 収束レート：$O(e^{-e\cdot t})$

また，次の２種類のGradient dominanceが成立します：

---

$$
\left\langle\frac{d \pi_\theta^{\top} r}{d \theta}, r\right\rangle \geq \pi_\theta\left(a^*\right) \cdot \Delta \cdot\left(\pi^*-\pi_\theta\right)^{\top} r .
$$

また，

$$
\left(\pi^{\prime}-\pi\right)^{\top} r \geq\left[1-\frac{1}{\pi\left(a^*\right) \cdot\left(e^{\eta \cdot \Delta}-1\right)+1}\right] \cdot\left(\pi^*-\pi\right)^{\top} r
$$

ここで，
$$
\pi^{\prime}(a):=\frac{\pi(a) \cdot e^{\eta \cdot r(a)}}{\sum_{a^{\prime}} \pi\left(a^{\prime}\right) \cdot e^{\eta \cdot r\left(a^{\prime}\right)}}
$$
としてます．

---

**コメント：**：論文にLemma 2と３がどう役に立ってるのかあんまりちゃんと書いてないかも．何がcontinuousで何がdiscreteなんだ？ただ１つ目はSoftmax PGよりマシなバウンドになってる．Softmax PGは$\left\|\frac{d \pi_{\theta_t}^{\top} r}{d \theta_t}\right\|_2^2$が必要だけど，NPGは$\left\langle\frac{d \pi_{\theta_t}^{\top} r}{d \theta_t}, r\right\rangle$でいけてる．

２番めのやつを使うと特にExponentialなバウンドが出せます：

$$
\begin{aligned}
\left(\pi^*\right. & \left.-\pi_{\theta_{t+1}}\right)^{\top} r=\left(\pi^*-\pi_{\theta_t}\right)^{\top} r-\left(\pi_{\theta_{t+1}}-\pi_{\theta_t}\right)^{\top} r \\
& \leq \frac{1}{\pi_{\theta_t}\left(a^*\right) \cdot\left(e^{\eta \cdot \Delta}-1\right)+1} \cdot\left(\pi^*-\pi_{\theta_t}\right)^{\top} r \\
& \leq \frac{1}{\pi_{\theta_1}\left(a^*\right) \cdot\left(e^{\eta \cdot \Delta}-1\right)+1} \cdot\left(\pi^*-\pi_{\theta_t}\right)^{\top} r \\
& \leq \frac{1}{\left[\pi_{\theta_1}\left(a^*\right) \cdot\left(e^{\eta \cdot \Delta}-1\right)+1\right]^t} \cdot\left(\pi^*-\pi_{\theta_1}\right)^{\top} r \\
& =\frac{\left(\pi^*-\pi_{\theta_1}\right)^{\top} r}{e^{c \cdot t}}
\end{aligned}
$$

特に３行目は

$$
\begin{aligned}
\pi_{\theta_{t+1}}\left(a^*\right) & =\frac{\pi_{\theta_t}\left(a^*\right) \cdot e^{\eta \cdot r\left(a^*\right)}}{\sum_a \pi_{\theta_t}(a) \cdot e^{\eta \cdot r(a)}} \\
& =\frac{\pi_{\theta_t}\left(a^*\right)}{\sum_a \pi_{\theta_t}(a) \cdot e^{-\eta \cdot \Delta(a)}} \\
& \geq \pi_{\theta_t}\left(a^*\right) . \quad(\Delta(a) \geq 0)
\end{aligned}
$$

を使いました．結局普通のPGより指数的に早いので，**exactな勾配が取れる場合には**NPGを使ったほうが良いです．

### Geometry-aware Normalized PG

[Leveraging Non-uniformity in First-order Non-convex Optimization](https://proceedings.mlr.press/v139/mei21a.html)をちゃんと読もう．

NPGはLSバウンドを良くする方向に進化してますが，Geometry-aware Normalized PGは普通のPGのLSバウンドのnon-uniformnessを利用しています．

* 更新方法：$\theta_{t+1} \leftarrow \theta_t+\eta \cdot \frac{d \pi_{\theta_t}^{\top} r}{d \theta_t} /\left\|\frac{d \pi_{\theta_t}^{\top} r}{d \theta_t}\right\|_2$


次が成立します：
$$
\left(\pi^*-\pi_{\theta_t}\right)^{\top} r \leq\left(\pi^*-\pi_{\theta_1}\right)^{\top} r \cdot e^{-\frac{c \cdot(t-1)}{12}}
$$




## 確率的更新の場合

On-policyな更新を考えましょう．次の重点サンプリング（IS）を使ってUnbiasedな更新をすることを考えます．
$a_t \sim \pi_{\theta_t}(\cdot)$について，
$$
\hat{r}_t(a)=\frac{\mathbb{I}\left\{a_t=a\right\}}{\pi_{\theta_t}(a)} \cdot r(a) \text { for all } a \in[K]
$$
とします．

さて，このISを使った更新をした場合，
* softmax PGは最適方策に確率収束します．
* NPGは
  * 正の確率で$\sum_{a \neq a^*} \pi_{\theta_t}(a) \rightarrow 1$に収束し
  * 正の確率で，任意の$a$について，$\pi_{\theta_t}(a) \rightarrow 1$に収束します．
* GNPGは正の確率で$\pi_{\theta_t}(a) \rightarrow 1$に収束します．

つまり，NPGとGNPGは確率的方策勾配を使った場合に最適方策を得ない場合があります．

なぜこんなことが起きるのか？の話はCommittal rateの章を読もう（第３章）．

## TODO: baselineについて

